Before you turn this problem in, make sure everything runs as expected. First, **restart the kernel** (in the menubar, select Kernel$\rightarrow$Restart) and then **run all cells** (in the menubar, select Cell$\rightarrow$Run All).

Make sure you fill in any place that says `YOUR CODE HERE` or "YOUR ANSWER HERE", as well as your name and collaborators below:

In [ ]:
NAME = "Ivan Dario Ovalle Benavides"
COLLABORATORS = ""

---

![Spark Logo](http://spark-mooc.github.io/web-assets/images/ta_Spark-logo-small.png)  ![Python Logo](http://spark-mooc.github.io/web-assets/images/python-logo-master-v3-TM-flattened_small.png)
# Contando palabras: Construye una aplicación que cuente palabras de forma eficiente

En este ejercicio de laboratorio se trabajarán las tecnologías descritas en los materiales del curso sobre Spark en base a una serie de ejercicios orientados al conteo de palabras. 

Con el uso masivo de Internet y las redes sociales, el volumen de texto no estructurado esta creciendo dramáticamente, y Spark es una gran herramienta para analizar este tipo de datos. En esta PEC, vamos a escribir código para encontrar las palabras mas comunes en un texto generado en latín, el ya conocido [Lorem Ipsum](https://www.lipsum.com/).


Lo mas interesante de la forma de trabajar en esta practica es que podría escalarse para analizar contenido web, por ejemplo, dentro de Wikipedia.

## Durante esta PEC vamos a cubrir:

* *Parte 1:* Creación de un RDD y un pair RDD <b>(5 Puntos)</b>
* *Parte 2:* Contar palabras usando un pair RDD <b>(8 Puntos)</b>
* *Parte 3:* Encontrar palabras concretas y su frecuencia de aparición media <b>(5 Puntos)</b>
* *Parte 4:* Aplicar las funcionalidades desarrolladas a un archivo de texto <b>(20 Puntos)</b>
* *Parte 5:* Calcular algunos estadísticos <b>(12 Puntos)</b>


> Como referencia a todos métodos que se requieren para implementar esta práctica podéis consultar:
> * [API Python de Spark](https://spark.apache.org/docs/2.4.0/rdd-programming-guide.html)

## Parte 1: Creación de un RDD y un pair RDDs

En esta sección, exploraremos como crear RRDs usando `parallelize` y como aplicar pair RDDs al problema del conteo de palabras.

### (0a) ¿Qué es un RDD (Resilient Distributed Dataset)?

Como hemos visto, un RDD es una colección de elementos tolerante a fallos capaz de operar en paralelo, con las siguientes características:

- Son inmutables
- Están distribuidos en los distintos nodos del clúster
- Son tolerantes a fallos

### (0b) Configuración del entorno python + spark

In [3]:
import findspark
findspark.init()

from pyspark import SparkConf, SparkContext

conf = SparkConf()
conf.setMaster("local[1]")


In [4]:
#replace "yourloginappname" by your UOC login and remove the ">" and "<" symbols
conf.setAppName("ivanovalle")
sc = SparkContext.getOrCreate(conf=conf)
sc.version

'2.4.0-cdh6.2.0'

### (1a) Creación de un RDD
Empezemos generando un RDD a partir de una lista de Python y el método `sc.parallelize`. Luego mostraremos por pantalla el tipo de la variable generada.

In [5]:
wordsList = ['lion', 'cat', 'tiger', 'elephant', 'rat', 'tiger', 'rat', 'cat']
wordsRDD = sc.parallelize(wordsList, 4)
# Print out the type of wordsRDD
type(wordsRDD)

pyspark.rdd.RDD

### (1b) Crear el plural de las palabas y testear (1 Punto)

Vamos a utilizar una transformación `map()` para incorporar la letra 's' a cada uno de los strings almacenados en el RDD que acabamos de crear. Primero definiremos una función de Python que devuelva la palabra que se le ha pasado como parámetro incorporando una "s" al final de esta. 

En este ejercicio, se pide reemplazar el texto `<FILL IN>` con la solución propuesta. Después de haber definido correctamente la función `makePlural`, ejecutar la siguiente celda que contiene una serie de asserts de test. Si la solución es correcta, se imprimirá `1 test passed`.

Esta será la forma habitual de trabajar en las PECs. Los ejercicios contendrán una explicación de lo que se espera, seguido de una celda de código con uno o más `<FILL IN>`. Las celdas que necesiten ser modificadas contendrán el texto `# TODO: Replace <FILL IN> with appropriate code` en la primera línea. En algunos casos, como veréis al final de la descripción del enunciado tendréis una pequeña ayuda (HINT) para completar el ejercicio.

Una vez se hayan sustituido todos los `<FILL IN>` por el código Python adecuado, ejecutar la celda, y posteriormente ejecutar la celda siguiente de test para comprobar que la solución es la esperada.

In [6]:
# TODO: Replace <FILL IN> with appropriate code
def makePlural(word):
    """Adds an 's' to `word`.

    Note:
        This is a simple function that only adds an 's'.  

    Args:
        word (str): A string.

    Returns:
        str: A string with 's' added to it.
    """
    #START <FILL IN>
    return word+"s"
    #END <FILL IN>

In [7]:
# TEST Pluralize and test (1b)
assert makePlural('lion')=='lions'
assert makePlural('cat')=='cats'
assert makePlural('tiger')=='tigers'

### (1c) Aplicar `makePlural` a nuestro RDD (1 Punto)

Ahora es el momento de aplicar nuestra función `makePlural()` a todos los elementos del RDD usando la transformación [map()](http://spark.apache.org/docs/2.4.0/api/python/pyspark.html#pyspark.RDD.map). Se pide completar la siguiente celda y posteriormente ejecutar la acción [collect()](http://spark.apache.org/docs/2.4.0/api/python/pyspark.html#pyspark.RDD.collect) en la celda de test para obtener el RDD transformado.

A continuación, se ofrece una pequeña ayuda.

**HINT (Copy & Paste and Complete):** 
```python
pluralRDD = wordsRDD.map(< FILL_IN >)
```

In [8]:
# TODO: Replace <FILL IN> with appropriate code
#START <FILL IN>
pluralRDD = wordsRDD.map(lambda x: makePlural(x))
pluralRDD.take(8)
#END <FILL IN>

['lions', 'cats', 'tigers', 'elephants', 'rats', 'tigers', 'rats', 'cats']

In [9]:
# TEST Apply makePlural to the base RDD(1c)
assert pluralRDD.collect() == ['lions', 'cats', 'tigers', 'elephants', 'rats', 'tigers', 'rats', 'cats']

### (1d) Aplicar una función `lambda` a lo largo de los valores en un RDD mediante la función `map` (1 Punto)

Ahora vamos a realizar una transformación equivalente a la anterior mediante una `lambda` función en lugar de una función con nombre.

Se pide completar la siguiente celda con la solución. Podéis basaros en el siguiente HINT, y los ejemplos que se muestran en [API Python de Spark](https://spark.apache.org/docs/2.4.0/rdd-programming-guide.html)

**HINT (Copy & Paste and Complete):** 
```python
pluralLambdaRDD = wordsRDD.map(< FILL IN >)
```

In [10]:
pluralLambdaRDD = wordsRDD.map(lambda x: x+"s")
pluralLambdaRDD.take(8)

['lions', 'cats', 'tigers', 'elephants', 'rats', 'tigers', 'rats', 'cats']

In [11]:
# TEST Cell: run collect action to the RDD (pluralLambdaRDD) obtained in the previous cell (1d)
assert pluralLambdaRDD.collect() == ['lions', 'cats', 'tigers', 'elephants', 'rats', 'tigers', 'rats', 'cats']

### (1e) Número de caracteres de cada una de las palabras (1 Punto)

Ahora se pide obtener el numero de caracteres de cada palabra mediante un transformación `map()` y una función `lambda`. Usaremos la acción `collect` para guardar este resultado directamente en una variable.

**HINT (Copy & Paste and Complete):** 
```python
pluralLengths = pluralRDD.map(< FILL_IN >).< FILL_IN >
```

In [12]:
pluralLengths = pluralRDD.map(lambda x:len(x)).collect()
print(pluralLengths)

[5, 4, 6, 9, 4, 6, 4, 4]


In [13]:
# TEST Length of each word (exercice 1e)
assert pluralLengths == [5, 4, 6, 9, 4, 6, 4, 4]

### (1f) Pair RDDs (1 Punto)

Para completar nuestro programa de conteo de palabras ahora vamos a trabajar con los llamados pair RDD. Un pair RDD es un RDD en el cual cada elemento es un tupla del estilo `(k, v)` donde `k` es la clave y `v` es su valor correspondiente. Primeramente se pide crear un pair RDD que contenga tuplas con el formato `('<word>', 1)` para cada elemento de nuestro RDD básico.

Podemos crear nuestro pair RDD usando una transformación `map()` y una función `lambda()` que transforme adecuadamente los elementos del RDD base.

**HINT (Copy & Paste and Complete):** 
```python
wordPairs = wordsRDD.map(< FILL IN >)
```

In [14]:
wordPairs = wordsRDD.map(lambda x:(x,1))
wordPairs.take(8)

[('lion', 1),
 ('cat', 1),
 ('tiger', 1),
 ('elephant', 1),
 ('rat', 1),
 ('tiger', 1),
 ('rat', 1),
 ('cat', 1)]

In [15]:
# TEST Pair RDDs (1f)
assert wordPairs.collect() == [('lion', 1), ('cat', 1), ('tiger', 1), ('elephant', 1), ('rat', 1), ('tiger', 1), ('rat', 1), ('cat', 1)]

## Parte 2: Contar palabras usando un pair RDD

Ahora, se pide contar el número de veces que cada palabra en particular aparece en el RDD. Esta operación se puede realizar de una infinidad de maneras, pero algunas serán mucho menos eficientes que otras.

Un solución muy sencilla sería usar `collect()` sobre todos los elementos devolverlos al driver y allí contarlos. Mientras esta forma de trabajar podría funcionar con textos relativamente cortos, nosotros lo que queremos es poder trabajar con textos de cualquier longitud. Adicionalmente, ejecutar todo el cálculo en el driver es mucho más lento que ejecutarlo en paralelo en los workers. Por estos motivos, en esta practica usaremos operaciones paralelizables.

### (2a) Usando `groupByKey()` (2 Puntos)
Una primera propuesta de solución a nuestro problema, luego veremos que hay otras mucho más eficientes, se podría basar en la transformación [groupByKey()](http://spark.apache.org/docs/2.4.0/api/python/pyspark.html#pyspark.RDD.groupByKey). Como su nombre indica, la transformación `groupByKey()` agrupa todos los elementos de un RDD que compartan la misma clave en una única lista dentro de una de las particiones.

Esta operación plantea dos problemas:
  + Necesita mover todos los valores dentro de la partición adecuada. Esto satura la red. 
  + Las listas generadas pueden llegar a ser muy grandes llegando incluso a saturar la memoria de alguno de los workers
  
Veremos como solucionar estos problemas posteriormente. De momento se pide usar transformación wide `groupByKey()` para generar un pair RDD del tipo `('word', iterator)`.

**HINT (Copy & Paste and Complete):**
```python
wordsGrouped = wordPairs.< FILL IN >
for key, value in wordsGrouped.collect():
    print('{0}: {1}'.format(key, list(value)))
```

In [16]:
wordsGrouped = wordPairs.groupByKey()
for key, value in wordsGrouped.collect():
    print('{0}: {1}'.format(key, list(value)))

lion: [1]
tiger: [1, 1]
cat: [1, 1]
elephant: [1]
rat: [1, 1]


In [17]:
# TEST groupByKey() approach (2a)
assert sorted(wordsGrouped.mapValues(lambda x: list(x)).collect()) == [('cat', [1, 1]), ('elephant', [1]), ('lion', [1]), ('rat', [1, 1]), ('tiger', [1, 1])]

### (2b) Utiliza `groupByKey()` para obtener los conteos (2 Puntos)

Usando la transformación `groupByKey()` crea un RDD que contenga 2 elementos, donde cada uno de ellos sea un par palabra (clave) iterador de Python (valor). Se puede reaprovechar el RDD `wordsGrouped`.

Luego suma todos los valores del iterador usando una transformación `map()`. El resultado debe ser un pair RDD que contenga las parejas (word, count).

**HINT (Copy & Paste and Complete):** 
```python
wordCountsGrouped = wordsGrouped.< FILL IN >
```

In [31]:
wordCountsGrouped = wordsGrouped.map(lambda x:(x[0],len(list(x[1]))))
wordCountsGrouped.collect()

[('lion', 1), ('tiger', 2), ('cat', 2), ('elephant', 1), ('rat', 2)]

In [32]:
# TEST Use groupByKey() to obtain the counts (2b)
assert sorted(wordCountsGrouped.collect())==[('cat', 2), ('elephant', 1), ('lion', 1), ('rat', 2), ('tiger', 2)]

### (2c) Conteo usando `reduceByKey` (2 Puntos)

Intentando solucionar la problemática citada anteriormente, una mejor solución para obtener los conteos es comenzar desde un pair RDD y luego usar la transformación [reduceByKey()](http://spark.apache.org/docs/2.4.0/api/python/pyspark.html#pyspark.RDD.reduceByKey) para crear un nuevo pair RDD. La transformación `reduceByKey()` agrupa todas las parejas que comparten la misma clave. Posteriormente aplica la función que se le pasa por parámetro agrupando los valores de dos en dos. Este proceso se repite iterativamente hasta que obtenemos un único valor agregado para cada una de las claves del pair RDD. La transformación `reduceByKey()` opera aplicando la función primero dentro de cada una de las particiones de forma independiente, y posteriormente solo comparte los valores agregados entre particiones diferentes, permitiéndole escalar de forma eficiente ya que no tiene necesidad de desplazar por la red una gran cantidad de datos.

**HINT (Copy & Paste and Complete):** 
```python
wordCounts = wordPairs.reduceByKey(< FILL IN >)
```

In [35]:
wordCounts = wordPairs.reduceByKey(lambda a,b:a+b)
wordCounts.collect()

[('lion', 1), ('tiger', 2), ('cat', 2), ('elephant', 1), ('rat', 2)]

In [34]:
# TEST Counting using reduceByKey (2c)
assert sorted(wordCounts.collect())==[('cat', 2), ('elephant', 1), ('lion', 1), ('rat', 2), ('tiger', 2)]

### (2d) Ahora todo junto (2 Puntos)

La version más compleja para solucionar el problema del conteo ejecuta primero un `map()` sobre el pair RDD, luego la transformación `reduceByKey()` y, finalmente, la acción `collect()` en una única linea de codigo.

**HINT (Copy & Paste and Complete):** 
```python
wordCountsCollected = (wordsRDD
    .< FILL IN >
    ...
    .< FILL IN >                       
    .collect())
```

In [38]:
wordCountsCollected = (wordsRDD
    .map(lambda x:(x,1))
    .reduceByKey(lambda a,b:a+b)                      
    .collect())
print(wordCountsCollected)

[('lion', 1), ('tiger', 2), ('cat', 2), ('elephant', 1), ('rat', 2)]


In [ ]:
# TEST exercise 2d
assert sorted(wordCountsCollected)==[('cat', 2), ('elephant', 1), ('lion', 1), ('rat', 2), ('tiger', 2)]

## Parte 3: Encontrar las palabras individuales y su frecuencia de aparición media

### (3a) Palabras únicas (2 Puntos)

Se pide obtener el número de palabras únicas en `wordsRDD`. Puedes utilizar otros RDDs que hayas creado en esta práctica si te resulta mas sencillo.

**HINT (Copy & Paste and Complete):** 
```python
uniqueWords = < FILL IN >
```

In [122]:
uniqueWords = wordCounts.distinct().count()
print(uniqueWords)

5


In [121]:
# TEST exercise 3a
assert uniqueWords== 5

### (3b) Calular la frecuencia media de aparición usando `reduce()` (3 Puntos)

Encuentra la frecuencia media de aparición de cada palabra en el RDD `wordCounts`.

Se pide utilizar la acción `reduce()` para sumar los conteos en `wordCounts` y entonces dividir por el número de palabras únicas. Para realizar esto primero aplica un `map()` al pair RDD `wordCounts`, que esta formado por tuplas con el formato (key, value), para convertirlo en un RDD de valores.

**HINT (Copy & Paste and Complete):** 

```python
from operator import add
totalCount = (wordCounts
    .map(< FILL IN >)
    .reduce(< FILL IN >))
average = < FILL IN > / float(< FILL IN >)
```

NOTA: en caso de resolver el problema siguiendo el proceso anterior el ejercicio se valorará en 2 puntos. Si se encuentra una solución que solo utilizen una sola instrucción para obtener el resultado se valorará en 3 puntos. Es decir, sin tener que aplicar la división en una segunda instrucción.

In [61]:
from operator import add
totalCount = (wordCounts
              .map(lambda x: x[1])
              .reduce(lambda a,b:a+b))
average = totalCount / float(wordCounts.groupByKey().count())
average

1.6

In [62]:
# TEST exercise 3b
assert round(average, 2)==1.6, 'incorrect value of average'

## Parte 4: Aplicar las funcionalidades desarrolladas a un archivo de texto

A este objetivo se propone construir una funcion `wordCount`, capaz de trabajar con datos reales, que suelen presentan problemas como el uso de mayúsculas o minúsculas, puntuación, acentos, etc. Posteriormente, se pide cargar los datos de nuestra fuente de datos y finalmente, calcular el conteo de palabras sobre los datos procesados.

### (4a) Función `wordCount` (4 Puntos)

Se pide definir una función para el conteo de dichas palabras. Se aconseja reusar las técnicas que has visto en los apartados anteriores de esta práctica. Dicha función, ha de tomar un RDD que contenga una lista de palabras, y devolver un pair RDD que contenga todas las palabras con sus correspondientes conteos.

In [63]:
# TODO: Replace <FILL IN> with appropriate code
def wordCount(wordListRDD):
    """Creates a pair RDD with word counts from an RDD of words.

    Args:
        wordListRDD (RDD of str): An RDD consisting of words.

    Returns:
        RDD of (str, int): An RDD consisting of (word, count) tuples.
    """
    #START <FILL IN>
    return wordListRDD.map(lambda x:(x,1)).reduceByKey(lambda a,b:a+b)
    #END <FILL IN>

In [64]:
# TEST exercise 4a
assert sorted(wordCount(wordsRDD).collect())==[('cat', 2), ('elephant', 1), ('lion', 1), ('rat', 2), ('tiger', 2)]

### (4b) Mayúsculas y puntuación (4 Puntos)

Los ficheros con datos reales son mucho más complejos que los que hemos estado usando en esta PAC. Algunos de los problemas a tratar son:
  + Las palabras deben de contarse independientemente de si contienen mayúsculas o minúsculas (por ejemplo, Spark y spark deberían contarse como la misma palabra).
  + Todos los signos de puntuación han de eliminarse.
  + Cualquier espacio al principio o al final de la palabra debe eliminarse.
  
Se pide definir la función `removePunctuation` que convierta todo el texto a minúsculas, elimine los signos de puntuación y los espacios al principio y fin de cada palabra. Usa el modulo de Python [re](https://docs.python.org/2/library/re.html) para eliminar cualquier carácter que no sea una letra, un número o un espacio.

Sino estas familiarizado con las expresiones regulares se aconseja revisar [este tutorial](https://developers.google.com/edu/python/regular-expressions). Alternativamente, [esta web](https://regex101.com/#python) es de gran ayuda para debugar tus expresiones regulares.

**Hints**

1. Usa la función [re.sub()](https://docs.python.org/2.7/library/re.html#re.sub).
2. Para nuestros propósitos, "puntuación" significa no alfabético, numérico, o espacio. La expresión regular que define estos caracteres es: `[^A-Za-z\s\d]`
3. No usar `\W`, ya que retendrá los guiones bajos.

In [71]:
# TODO: Replace <FILL IN> with appropriate code
import re
def removePunctuation(text):
    """Removes punctuation, changes to lower case, and strips leading and trailing spaces.

    Note:
        Only whitespace, letters, and numbers should be retained.  Other characters should should be
        eliminated (e.g. it's becomes its).  Leading and trailing spaces should be removed after
        punctuation is removed.

    Args:
        text (str): A string.

    Returns:
        str: The cleaned up string.
    """
    #START <FILL IN>
    text = re.sub(r'[^A-Za-z\s\d]', '', text)
    text = text.lower()
    text = text.strip()
    return text
    #END <FILL IN>
  
print(removePunctuation('Hi, you!'))
print(removePunctuation(' No under_score!'))
print(removePunctuation(' *      Remove punctuation then spaces  * '))

hi you
no underscore
remove punctuation then spaces


In [72]:
# TEST exercise 4b
assert removePunctuation(" The Elephant's 4 cats. ") == 'the elephants 4 cats'

### (4c) Cargar un fichero de texto

A partir de este punto usaremos el texto ya mencionado Lorem Ipsum generado para la práctica. Para cargar un fichero de texto a un RDD, usaremos el método `SparkContext.textFile()`. Una vez cargado el fichero se pide limpiar los datos mediante `removePunctuation()` dentro de una transformación `map()`. Dado que el fichero es bastante grande, usaremos `take(15)`, de forma que tan solo imprimiremos por pantalla las 15 primeras líneas.

Se pide revisar y ejecutar la siguiente celda.

In [73]:
# Just run the code
import os.path

fileName = os.path.join('/aula_M2.858/data/', 'LoremIpsum.txt')

loremRDD = sc.textFile(fileName, 8).map(removePunctuation).filter(lambda x: len(x)>0)
loremRDD.take(10)

['aut minima deleniti et autem minus illo esse dolores eligendi corrupti dolore minima nostrum eos nobis nam nihil aspernatur nam ut quae sint laborum ut dolores error possimus aperiam consequatur',
 'pariatur sed quo non itaque qui pariatur saepe ad quis consequatur nihil iste molestias et eos ut expedita vel reiciendis dolorem enim doloribus quam architecto aperiam',
 'sed repudiandae pariatur similique est aut sequi animi in aperiam enim ipsa enim dolorem inventore aut quo odio in consequatur et',
 'aspernatur ad esse et aliquid itaque dolores rerum quia commodi explicabo non magnam nostrum consectetur non sint eum nulla et aut quis doloribus itaque nulla molestiae quis est est quo facilis incidunt a ipsa in itaque sed aut nobis facere dignissimos atque unde cum ea vero',
 'tenetur vel quod voluptatum laudantium dolores neque aut est modi qui aperiam itaque aperiam quae ratione doloremque aut delectus quas qui',
 'qui placeat vel ipsam praesentium sint recusandae dicta minus praesen

### (4d) Extraer las palabras de las lineas (4 Puntos)

Antes de poder usar la función `wordcount()`, hemos de solucionar dos problemas con el formato del RDD:
  + El primer problema es que necesitamos obtener las palabras que contiene cada línea. Esto lo solucionaremos en el apartado (4d).
  + El segundo problema es que necesitamos filtrar las líneas completamente vacías. Esto lo solucionaremos en el apartado (4e).

Para aplicar una transformación que divida cada elemento del RDD por sus espacios, se propone aplicar la función incorporada en los strings de Python [split()](https://docs.python.org/2/library/string.html#string.split). Cuidado que a primera vista puede parecer que la función necesaria es una transformacion `map()`, pero si analizas el resultado de la función `split()` te darás cuenta que esta no es la opción correcta.

> Nota:
> * No uses la implementación estándar del `split()`, pásale un valor de separación. Por ejemplo, para dividir `line` por comas, usa `line.split(',')`.

**HINT (Copy & Paste and Complete):** 
```python
loremWordsRDD = loremRDD.< FILL IN >
```

In [86]:
# TODO: Replace <FILL IN> with appropriate code
#START <FILL IN>
# YOUR CODE HERE
loremWordsRDD = loremRDD.flatMap(lambda x: x.split(" "))
#END <FILL IN>
loremWordsCount = loremWordsRDD.count()
loremWordsCount

29249699

In [85]:
# TEST Words from lines (4d)
# This test allows for leading spaces to be removed either before or after
# punctuation is removed.
assert loremWordsCount == 29249699
assert loremWordsRDD.top(5)==['voluptatum', 'voluptatum', 'voluptatum', 'voluptatum', 'voluptatum']

### (4e) Mejorando la eficiencia mediante persistencia (2 Puntos)

Una vez que hemos contado el número de palabras con el método `count()`, nos vamos a centrar en hacer el código más eficiente. 

Para reducir los tiempos de ejecución, vamos a utilizar la persistencia de un RDD `persist()`, que principalmente se utiliza cuando se realizan múltiples operaciones repetidas sobre un RDD. Cuando se utiliza la persistencia, cada nodo conserva la partición del RDD que opera en la memoria.

El buen uso de la persistencia de un RDD puede llegar a aumentar el rendimiento de las aplicaciones Spark en más de 10 veces. Para algoritmos iterativos y aplicaciones interactivas rápidas, la persistencia es de vital importancia.

Existen diferentes niveles de persistencia, dependiendo de si se almacena el RDD en memoria o en disco. En este ejemplo utilizaremos el nivel de persistencia por defecto MEMORY_ONLY.

Mediante las siguientes celdas vamos a poder ver las ventajas de la persistencia. Para poder cuantificar la mejora del rendimiento nos ayudaremos de la función `%%time`, que imprime el CPU time y el wall clock time de ejecución para la celda completa.

> Nota:
> * Wall clock time: tiempo total real que lleva ejecutar una tarea (CPU time, I/O time y tiempo de comunicación entre máquinas si los datos están repartidos entre varias)
>* CPU time: tiempo total que está la CPU ejecutando una tarea

Siguientemente haremos una serie de ejecuciones sobre un RDD utilizando el método `count()` aunque podríamos utilizar diferentes métodos o combinaciones de métodos, pero lo haremos así por simplicidad.

1. Contamos el número de palabras y vemos el tiempo invertido en la ejecución

In [87]:
%%time
loremWordsRDD.count()

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 15 s


29249699

2. Se pide realizar un `persist()` sobre el RDD `loremWordsRDD` (1 Punto)

**HINT (Copy & Paste and Complete):** 
```python 
loremWordsRDD.< FILL IN >
```

In [88]:
# TODO: Complete the cell with the appropriate code
loremWordsRDD.persist()

PythonRDD[102] at RDD at PythonRDD.scala:53

3. Volvemos a contar el número de palabras después de hacer el `persist()` y observamos que el tiempo de ejecución es mayor que antes de hacer el `persist()` ya que debe almacenar los resultados en los nodos para su posterior uso.

In [89]:
%%time
loremWordsRDD.count()

CPU times: user 16 ms, sys: 0 ns, total: 16 ms
Wall time: 22.9 s


29249699

4. En esta segunda ejecución del `count()` y posteriores observaremos que el tiempo de ejecución es casi 10 veces menor

In [90]:
%%time
loremWordsRDD.count()

CPU times: user 12 ms, sys: 0 ns, total: 12 ms
Wall time: 4.56 s


29249699

5. Si queremos des-persistir el RDD que hemos persistido con `persist()` utilizaremos `unpersist()` (1 Punto)


**HINT (Copy & Paste and Complete):** 
```python
loremWordsRDD.< FILL IN >
```

In [91]:
# TODO: Replace <FILL IN> with appropriate code
#START <FILL IN>
loremWordsRDD.unpersist()
#END <FILL IN>

PythonRDD[102] at RDD at PythonRDD.scala:53

6. Volvemos a contar el número de palabras con el `count()` y observamos que volvemos a tener tiempos de ejecución similares a antes de ejecutar el `persist()`

In [92]:
%%time
loremWordsRDD.count()

CPU times: user 8 ms, sys: 4 ms, total: 12 ms
Wall time: 15.4 s


29249699

### (4f) Calcula palabras distintas (3 Puntos)

En este punto se pide contar cuantas palabras distintas contiene nuestro texto


**HINT (Copy & Paste and Complete):** 
```python
distintWordsMapRDD = loremWordsRDD.< FILL IN >
```

In [107]:
# TODO: Replace the following block with appropriate code
#START <FILL IN>
distintWordsMapRDD = loremWordsRDD.map(lambda x: (x,1))
#END <FILL IN>
distintWordsRDD=distintWordsMapRDD.keys().distinct()

In [98]:
# TEST Remove empty elements (4f)
assert distintWordsRDD.count()== 182

### (4g) Cuenta las palabras (3 Puntos)

Ahora que tenemos un RDD que contiene solo palabras. Se pide aplicar la función `wordCount()` para producir una lista con los conteos de palabras. Podemos ver las 15 más comunes usando la acción `takeOrdered()`; sin embargo, como los elementos del RRD son pares, necesitamos una función especial que ordene los pares de la forma correcta.


Usa las funciones  `wordCount()` y `takeOrdered()` para obtener las 15 palabras más comunes junto con sus conteos.

**HINT (Copy & Paste and Complete):** 
```python
top15WordsAndCounts = wordCount(loremWordsRDD).< FILL IN >
```

In [104]:
# TODO: Complete the cell with the appropriate code
top15WordsAndCounts = wordCount(loremWordsRDD).takeOrdered(15, key = lambda x: -x[1])
top15WordsAndCounts

[('et', 1291567),
 ('aut', 705322),
 ('ut', 704966),
 ('qui', 704703),
 ('est', 587782),
 ('voluptatem', 469634),
 ('quia', 469401),
 ('rerum', 353054),
 ('sed', 352873),
 ('omnis', 352653),
 ('consequatur', 352436),
 ('sit', 352396),
 ('non', 351959),
 ('voluptas', 351127),
 ('dolorem', 235822)]

In [105]:
# TEST Count the words (exercise 4g)
assert top15WordsAndCounts== [('et', 1291567), ('aut', 705322), 
                              ('ut', 704966), ('qui', 704703), 
                              ('est', 587782), ('voluptatem', 469634), 
                              ('quia', 469401), ('rerum', 353054), ('sed', 352873),
                              ('omnis', 352653), ('consequatur', 352436), 
                              ('sit', 352396), ('non', 351959), ('voluptas', 351127), 
                              ('dolorem', 235822)]

## Parte 5: Calcular algunos estadísticos

Usando las mismas técnicas que has aplicado en los ejercicios anteriores responde a las siguientes preguntas:

- ¿Cuantas palabras distintas tienen exactamente dos 's'? (4 Puntos)
- ¿Cual es la palabra de siete letras que más se repite? ¿Cuantas veces aparece? (4 Puntos)
- ¿Cuantas palabras distintas tienen más consonantes que vocales? (4 Puntos)

In [108]:
#¿Cuantas palabras distintas tienen exactamente dos 's'?
def two_s(word):
    if word.count("s")==2:
        return True
    else:
        return False

loremWordsRDD_2s = distintWordsRDD.filter(lambda x: two_s(x))
loremWordsRDD_2s.count()

7

In [109]:
# TEST Different words with exactly two 's' (5a)
assert loremWordsRDD_2s.count()== 7

In [116]:
#¿Cual es la palabra de siete letas que más se repite?¿Cuantas veces aparece?

def seven_letters(word):
    if len(word)==7:
        return True
    else:
        return False
    
distintWordsRDD_7_Number_Repetitions = wordCount(loremWordsRDD).filter(lambda x: seven_letters(x[0]))
distintWordsRDD_7_Number_Repetitions.takeOrdered(1, key = lambda x: -x[1])

[('dolorem', 235822)]

In [117]:
# TEST 7 letters most repeated word, how many times is repeated (5b)
assert distintWordsRDD_7_Number_Repetitions.count()== 23

In [118]:
#¿Cuantas palabras distintas tienen más consonantes que vocales?
vowels=['a','e','i','o','u']

def more_consonants_than_vowels(word):
    c_vowels=0
    c_consonants=0
    for letter in word:
        if letter in vowels:
            c_vowels=c_vowels+1
        else:
            c_consonants=c_consonants+1
    if c_consonants>c_vowels:
        return True
    else:
        return False

moreConstWordsRDD = distintWordsRDD.filter(lambda x: more_consonants_than_vowels(x))
moreConstWordsRDD.take(10)

['tempora',
 'porro',
 'consectetur',
 'debitis',
 'sunt',
 'ipsum',
 'fugit',
 'consequatur',
 'aspernatur',
 'nulla']

In [119]:
# TEST More consonants than vowels (5c)
assert moreConstWordsRDD.count()== 89